In [77]:
import psycopg2
import os
import requests
import time
from dotenv import load_dotenv

In [78]:
load_dotenv()

TEST_PUUID = 'cgchml5db7FVLW6cfygY7dmICGAp6hU6OPeRX809alh99kdQW0m1I_Uy_RbVbNj0cB6EJZGL0K_5ew'
TEST_MATCH_ID = 'NA1_5031446976'
RATE_LIMITS_PER_SECOND = 20
RIOT_API_KEY = os.getenv("RIOT_API_KEY")

DEFAULT_REGION_CODE = 'na1'
DEFAULT_REGION = 'americas'
INITIAL_PUUID = "iThpGepRyBenabNTRWRNBejzpNqlCDrosKcV9y3GINND7TDv9GcXqvySCtcABfIx48hHEW6FBZv1hg"

In [79]:
def create_connection():
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('5432')
    )

    cursor = conn.cursor()

    return conn, cursor

In [80]:
def create_match_data_table():
    #create connection to db
    conn, cursor = create_connection()

    table_check_sql = """
    SELECT EXISTS (
        SELECT FROM pg_tables 
        WHERE schemaname = 'public' 
        AND tablename = 'match_data'
    );
    """

    create_table_sql = """
            CREATE TABLE match_data (
                id SERIAL PRIMARY KEY,
                match_id TEXT NOT NULL,
                region TEXT NOT NULL,
                game_mode TEXT NOT NULL,
                elo TEXT NOT NULL,
                version TEXT NOT NULL,
                blue_one TEXT NOT NULL,
                blue_two TEXT NOT NULL,
                blue_three TEXT NOT NULL,
                blue_four TEXT NOT NULL,
                blue_five TEXT NOT NULL,
                red_one TEXT NOT NULL,
                red_two TEXT NOT NULL,
                red_three TEXT NOT NULL,
                red_four TEXT NOT NULL,
                red_five TEXT NOT NULL,
                winning_team INTEGER NOT NULL CHECK (winning_team IN (0, 1)),
                UNIQUE (match_id)
            );
            """
    try:
        cursor.execute(table_check_sql)
        table_exists = cursor.fetchone()[0]
        if not table_exists:
            cursor.execute(create_table_sql)
            conn.commit()
            print("Table created successfully")
        else:
            print("Table exists")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    cursor.close()
    conn.close()

In [81]:
def get_user_matches(puuid = None, region=DEFAULT_REGION):
    if not puuid:
        raise Exception('Please Select puuid')
    
    api_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids'
    headers = {
        'X-Riot-Token' : RIOT_API_KEY,
    }

    try:
        response = requests.get(api_url,headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error Occured: {e}')

In [82]:
def get_match_data(match_id = None, region=DEFAULT_REGION):
    if not match_id:
        raise Exception('Please Select Match ID')
    
    api_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}'
    headers = {
        'X-Riot-Token' : RIOT_API_KEY,
    }

    try:
        response = requests.get(api_url,headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error Occured: {e}')

In [83]:
def get_elo(puuid, region=DEFAULT_REGION_CODE):
    if not puuid:
        raise Exception('Please Select Match ID')
    
    get_summoner_url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}'
    headers = {
        'X-Riot-Token' : RIOT_API_KEY,
    }    
    try:
        summoner_response = requests.get(get_summoner_url,headers=headers)
        summoner_response.raise_for_status()
        summoner_response = summoner_response.json()['id']

        get_rank_url = f'https://{region}.api.riotgames.com/lol/league/v4/entries/by-summoner/{summoner_response}'
        rank_response = requests.get(get_rank_url,headers=headers)
        rank_response.raise_for_status()
        return rank_response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error Occured: {e}')


In [84]:
def put_data(match_id, match_data):
    conn, cursor = create_connection()

    insert_sql = """
    INSERT INTO match_data (
        match_id, region, game_mode, elo, version,
        blue_one, blue_two, blue_three, blue_four, blue_five,
        red_one, red_two, red_three, red_four, red_five,
        winning_team
    ) VALUES (
        %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s,
        %s
    )
"""

    region = match_data['info']['platformId']
    game_mode = match_data["info"]["gameMode"]
    version = match_data["info"]["gameVersion"]
    team = [match_data['info']['participants'][i]['championName'] for i in range(10)]
    blue_one, blue_two, blue_three, blue_four, blue_five, red_one, red_two, red_three, red_four, red_five = team
    winning_team =  0 if match_data['info']['participants'][0]['win'] else 1

    elo_data = get_elo(match_data['metadata']['participants'][0])
    if elo_data:
        for entry in elo_data:
            if entry.get('queueType') == 'RANKED_SOLO_5x5':
                elo = f"{entry['tier']} {entry['rank']}"
                break
            elif entry.get('queueType') == 'RANKED_FLEX_5x5':
                elo = f"{entry['tier']} {entry['rank']}"
                break
        else:
            elo = "UNRANKED"
    else:
        elo = "UNRANKED"
    
    try:
        cursor.execute(insert_sql,(
            match_id, region, game_mode, elo, version,
            blue_one, blue_two, blue_three, blue_four, blue_five,
            red_one, red_two, red_three, red_four, red_five,
            winning_team
        ))
        conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)

    conn.close()
    cursor.close()    
    

In [85]:
def main(initial_puuid = None):
    #create match data if it does not exist
    create_match_data_table()
    
    unique_users_set = {initial_puuid}
    unique_users_list = list(unique_users_set)
    current_index = 0

    while current_index <= 2500:
        cid = cid = unique_users_list[current_index]
        current_index+=1

        if(current_index % 100 == 0):
            print(current_index)

        try:
            match_history = get_user_matches(cid)
            for match_id in match_history:
                match_data = get_match_data(match_id)
                put_data(match_id, match_data)
                players = match_data['metadata']['participants']
                for player in players:
                    if player not in unique_users_set:
                        unique_users_set.add(player)
                        unique_users_list.append(player)
                time.sleep(0.5)
        except requests.exceptions.RequestException as e:
            print(f'Failed to retrieve data for user {cid}: {e}')
        


main(INITIAL_PUUID)

Table exists
1
duplicate key value violates unique constraint "match_data_match_id_key"
DETAIL:  Key (match_id)=(NA1_5035365410) already exists.

2
duplicate key value violates unique constraint "match_data_match_id_key"
DETAIL:  Key (match_id)=(NA1_5035375280) already exists.



KeyboardInterrupt: 